In [1]:
%pip install "ray[serve]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 MB 36.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 15.8 MB/s eta 0:00:00
  Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.4/367.4 kB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 65.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 42.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.8/897.8 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━

In [6]:
from focoos import Focoos, FocoosEnvHostUrl
from pprint import pprint
import os

focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.DEV)

pprint(focoos.list_models())

[11/08 16:37][INFO][focoos]: Currently logged as: fabio.cermelli@focoos.ai environment: https://api.dev.focoos.ai/v0


[{'description': 'A model for bottles detection',
  'focoos_model': 'focoos_object365',
  'name': 'lattuce',
  'ref': '20d5eaba14244600',
  'status': 'DEPLOYED',
  'task': 'detection'},
 {'description': 'A model for bottles detection',
  'focoos_model': 'focoos_object365',
  'name': 'bottles_2',
  'ref': '6ba0958840014109',
  'status': 'TRAINING_COMPLETED',
  'task': 'detection'},
 {'description': 'Test model for blister dataset',
  'focoos_model': 'focoos_ade_ins_medium',
  'name': 'blister2',
  'ref': 'ce656877a08f469d',
  'status': 'TRAINING_COMPLETED',
  'task': 'instseg'},
 {'description': 'Test model for boxes detection dataset',
  'focoos_model': 'focoos_object365',
  'name': 'boxes2',
  'ref': '43f95a71c20a4033',
  'status': 'TRAINING_COMPLETED',
  'task': 'detection'},
 {'description': 'Test model for concrete dataset',
  'focoos_model': 'focoos_ade_ins_medium',
  'name': 'concrete2',
  'ref': '8ab394c165044271',
  'status': 'TRAINING_COMPLETED',
  'task': 'instseg'},
 {'descr

In [7]:
import requests
import time

ray_address = "http://localhost:65036"

def ray_infer(model_id: str):
    image_path = "./assets/ade_val_034.jpg"
    im_bytes = open(image_path, "rb").read()
    response = requests.post(
        ray_address,
        data=im_bytes,
        headers={"serve_multiplexed_model_id": str(model_id)},
    )
    return response.json()


t0 = time.time()
res = ray_infer("focoos_object365")
t1 = time.time()
print(f"t: {t1-t0:.3f} seconds")
print(res)

JSONDecodeError: Extra data: line 1 column 4 (char 3)

In [ ]:
response = requests.post(
        ray_address,
        data={}
        
    )

In [ ]:
iters = 10

print("RAY")
for model_id in [
    "645b15a6b2c044c2",
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_object365",
    "focoos_aeroscapes",
    "focoos_ade_small",
    "focoos_isaid_nano",
    "focoos_coco_tiny",
]:
    t0 = time.time()
    for i in range(iters):
        ray_infer(model_id)
    t1 = time.time()
    print(
        f"{model_id} fps: {iters/(t1-t0):.3f}  ({(t1-t0)*1000/iters:.3f}ms per image)"
    )

In [ ]:
import logging
from focoos import Focoos, FocoosEnvHostUrl
from pprint import pprint
import os

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
iters = 10
focoos = Focoos(api_key=os.getenv("FOCOOS_API_KEY"), host_url=FocoosEnvHostUrl.LOCAL)
image_path = "./assets/ade_val_034.jpg"

print("API+TRITON+WAF")
prints = []
for model_id in [
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_object365",
    "focoos_aeroscapes",
    "focoos_ade_small",
    "focoos_isaid_nano",
]:
    model = focoos.get_model(model_id)

    t0 = time.time()
    for i in range(iters):
        model.remote_infer(image_path)
    t1 = time.time()
    _str = (
        f"{model_id} fps: {iters/(t1-t0):.3f}  ({(t1-t0)*1000/iters:.3f}ms per image)"
    )
    prints.append(_str)

print("\n".join(prints))

In [8]:
import ray

ray_address = "http://localhost:65036"
ray.init(address=ray_address)

2024-11-08 16:38:29,067	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


AssertionError: Module: http does not have ClientBuilder.

## RAY SERVE DEPLOY 

In [11]:
!serve deploy ../ray-config.yaml -a http://127.0.0.1:65036

2024-11-08 16:40:34,702	INFO scripts.py:243 -- Deploying from config file: '../ray-config.yaml'.
2024-11-08 16:40:34,853	SUCC scripts.py:350 -- 
Sent deploy request successfully.
 * Use `serve status` to check applications' statuses.
 * Use `serve config` to see the current application config(s).



In [ ]:
!serve status

In [ ]:
!serve config


In [ ]:
!serve shutdown -y

## RAY JOBS

In [23]:
!ray job submit --address http://localhost:8265 --runtime-env-json '{"container": {"image": "ghcr.io/focoosai/deeplearning:base-cu12-pt22"}}' nvidia-smi

I0000 00:00:1731084189.620124 4292219 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Job submission server address: http://localhost:8265

-------------------------------------------------------
Job 'raysubmit_KjUThDQxfRhqdgHD' submitted successfully
-------------------------------------------------------

Next steps
  Query the logs of the job:
    ray job logs raysubmit_KjUThDQxfRhqdgHD
  Query the status of the job:
    ray job status raysubmit_KjUThDQxfRhqdgHD
  Request the job to be stopped:
    ray job stop raysubmit_KjUThDQxfRhqdgHD

Tailing logs until the job exits (disable with --no-wait):

---------------------------------------
Job 'raysubmit_KjUThDQxfRhqdgHD' failed
---------------------------------------

Status message: runtime_env setup failed: Failed to set up runtime environment.
Could not create the actor because its associated runtime env failed to be created.
Traceback (most recent call last):
  File "/home/ray/anaconda3/lib/python3.9/site-packages/ray/_private/runtime_env/utils.py", line 80, in check_output_cmd
    proc = await asyncio.create_subpr

In [1]:
ray_address = "http://localhost:8265"

In [27]:
from ray.job_submission import JobSubmissionClient
from ray.runtime_env import RuntimeEnv
import os
import time

client = JobSubmissionClient(ray_address)

runtime_env = RuntimeEnv(
    env_vars={"FOCOOS_API_KEY": os.environ["FOCOOS_API_KEY"]},
    pip=[
       "onnxruntime-gpu==1.18.0 --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/"
        "requests",
        "Pillow~=10.4.0",
        "supervision~=0.24.0",
        "torch~=2.1.1",
        "opencv-python-headless~=4.9.0",
    ],
)
job_id = client.submit_job(
        entrypoint="python -c 'import torch; model = torch.hub.load(\"ultralytics/yolov5\", \"yolov5s\"); img = torch.zeros((1, 3, 640, 640)); results = model(img); print(results)'",
        runtime_env=runtime_env,
        entrypoint_num_gpus=0.3,
        submission_id=f"nvidia_smi_{int(time.time())}",
    )
print(client.get_job_status(job_id))
time.sleep(5)
print(client.get_job_status(job_id))
print(client.get_job_logs(job_id))

PENDING
PENDING



In [16]:
client.get_job_logs(job_id)

''

In [8]:
import ray

# Starting the Ray client. This connects to a remote Ray cluster.
ray.init(ray_address)
@ray.remote
def test():
    print("test")

2024-11-08 16:50:57,696	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver


KeyboardInterrupt: 

In [ ]:
from ray.job_submission import JobSubmissionClient
from ray.runtime_env import RuntimeEnv
import os


client = JobSubmissionClient("http://localhost:8265")
runtime_env = RuntimeEnv(
    # container={"image": "ghcr.io/focoosai/deeplearning:base-cu12-pt22"},
    pip=[
        "focoos",
        "requests",
        "Pillow~=10.4.0",
        "supervision~=0.24.0",
        "pydantic~=2.8.2",
        "pydantic-settings~=2.5.0",
        "tqdm~=4.66.6",
        "numpy~=1.26.4",
        "scipy~=1.14.1",
        "gradio",
    ],
    env_vars={"FOCOOS_API_KEY": os.environ["FOCOOS_API_KEY"]},
    # working_dir="./assets/"
)
for model_id in [
    "focoos_object365",
    "focoos_rtdetr",
    "focoos_ade_medium",
    "focoos_ade_small",
    "focoos_isaid_nano",
]:
    job_id = client.submit_job(
        entrypoint=f"python /app/apps/benchmark.py --model {model_id}",
        runtime_env=runtime_env,
        entrypoint_num_gpus=0.3,
        submission_id=f"{model_id}_benchmark_{int(time.time())}",
    )
    print(client.get_job_info(job_id))
    print(client.get_job_logs(job_id))
    print(client.get_job_status(job_id))

In [ ]:
print(client.get_job_status(job_id))
print(client.get_job_logs(job_id))

In [ ]:
client.get_job_info(job_id)

In [ ]:
client.delete_job("03000000")